# 🎬 Complete Movie Trailer Generator - Rule-Based (MacBook Air M2 Friendly)

In [1]:
# 📦 Install required libraries
!pip install opencv-python numpy ffmpeg-python librosa

In [2]:

import cv2
import numpy as np
import os

def extract_keyframes(video_path, output_folder, min_frame_gap=100):
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    frame_count = 0
    last_saved = -min_frame_gap  # so first frame passes
    saved_frames = []

    os.makedirs(output_folder, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray)
            mean_diff = np.mean(diff)

            # Save only if frame is different and far from last one saved
            if mean_diff > 25 and (frame_count - last_saved) >= min_frame_gap:
                frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
                cv2.imwrite(frame_filename, frame)
                saved_frames.append(frame_count)
                last_saved = frame_count

        prev_frame = gray
        frame_count += 1

    cap.release()
    print(f"Extracted {len(saved_frames)} spaced keyframes.")
    return saved_frames


In [3]:
# 🎞️ Step 2: Generate 2-Second Clips from Selected Frames
import subprocess

def generate_clips(video_path, frames, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    for frame in frames:
        timestamp = frame / fps
        clip_name = os.path.join(output_folder, f"frame_{frame}.mp4")
        command = f"ffmpeg -y -ss {timestamp} -t 2 -i {video_path} -vf scale=640:360 -preset veryfast -crf 28 -c:v libx264 -c:a aac {clip_name}"


        subprocess.run(command, shell=True)
    print("Clips generated.")

In [4]:
# 🎬 Step 3: Merge Clips into Final Trailer
def create_trailer(clips_folder, frames, output_trailer):
    list_file = os.path.join(clips_folder, "input_clips.txt")
    with open(list_file, "w") as f:
        for frame in frames:
            clip_path = f"frame_{frame}.mp4"
            f.write(f"file '{clip_path}'\n")

    output_path = os.path.join(clips_folder, output_trailer)
    command = f"ffmpeg -f concat -safe 0 -i {list_file} -c:v libx264 -preset fast -crf 23 -c:a aac -strict experimental {output_path}"

    subprocess.run(command, shell=True)
    print(f"Trailer created at: {output_path}")

In [10]:
# ▶️ Run the entire pipeline
video_file = "/content/VIDEO-2025-03-10-00-06-48.mp4"  # Replace with your movie file
keyframe_folder = "keyframes"
clips_folder = "clips"

print("Extracting keyframes...")
keyframes = extract_keyframes(video_file, keyframe_folder, min_frame_gap=100)


import random

selected_frames = random.sample(keyframes, min(40, len(keyframes)))  # 75 clips × 2s = 150s = 2.5 minutes

#selected_frames.sort()  # for smooth viewing



generate_clips(video_file, selected_frames, clips_folder)
create_trailer(clips_folder, selected_frames, "final_trailer.mp4")


Extracting keyframes...
Extracted 110 spaced keyframes.
Clips generated.
Trailer created at: clips/final_trailer.mp4


In [9]:
import os
import subprocess

def check_clip_durations(clips_folder):
    for file in os.listdir(clips_folder):
        if file.endswith(".mp4"):
            path = os.path.join(clips_folder, file)
            cmd = f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 \"{path}\""
            result = subprocess.getoutput(cmd)
            print(f"{file}: {result.strip()} sec")

check_clip_durations("clips")


frame_27070.mp4: 3.000000 sec
frame_45144.mp4: 3.000000 sec
frame_7577.mp4: 3.000000 sec
frame_66920.mp4: 3.000000 sec
frame_67383.mp4: 3.000000 sec
frame_9752.mp4: 3.000000 sec
frame_11393.mp4: 3.000000 sec
frame_68.mp4: 3.000000 sec
frame_4474.mp4: 3.000000 sec
frame_10665.mp4: 3.000000 sec
frame_14698.mp4: 3.000000 sec
frame_5808.mp4: 3.000000 sec
frame_58101.mp4: 3.000000 sec
frame_6928.mp4: 3.000000 sec
frame_10079.mp4: 3.000000 sec
frame_77476.mp4: 3.000000 sec
frame_2267.mp4: 3.000000 sec
frame_14620.mp4: 3.000000 sec
frame_2671.mp4: 3.000000 sec
frame_7706.mp4: 3.000000 sec
frame_27860.mp4: 3.000000 sec
frame_70051.mp4: 3.000000 sec
frame_6534.mp4: 3.000000 sec
frame_64568.mp4: 3.000000 sec
frame_8243.mp4: 3.000000 sec
frame_9535.mp4: 3.000000 sec
frame_12647.mp4: 3.000000 sec
frame_73514.mp4: 3.000000 sec
frame_39762.mp4: 3.000000 sec
frame_35058.mp4: 3.000000 sec
frame_10864.mp4: 3.000000 sec
frame_30171.mp4: 3.000000 sec
frame_51721.mp4: 3.000000 sec
frame_4915.mp4: 3.000000